# 📊 Análisis de Ofertas Laborales en Ecuador
Este notebook carga, limpia y analiza un dataset de ofertas de trabajo en Ecuador.
Incluye tablas, gráficos.

## 1. Cargar librerías y leer el dataset

In [68]:
#Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar visualización
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

# Cargar el dataset
ruta = "../Datos/oferta_laboral_ecuador.csv"  # Ajustado a tu estructura
df = pd.read_csv(ruta)

# Vista previa de los datos
df.head()


,cargo,modo,fechaPublicado,fechaFin,plazas,experiencia,capacitacion,jornadas,remuneracion,nivelInstruccion,areaEstudios,ciudad,parroquia,sector
0,desarrollador java jee,tiempo completo,2013-02-22,2013-03-24,1,sin experiencia,0-50 horas,jornada ordinaria (8 horas),$501-$750,tercer nivel,informática software,quito,indistinto,norte
1,guardias de seguridad con experiencia 1 año,tiempo completo,2013-02-23,2013-02-28,20,7-12 meses,0-50 horas,jornada ordinaria (8 horas),$400-$500,bachiller,recursos humanos/personal,ibarra,indistinto,centro
2,vendedor,tiempo completo,2013-02-23,2013-02-26,2,1-3 años,0-50 horas,jornada ordinaria (8 horas),$400-$500,bachiller,ventas al consumidor,guayaquil,indistinto,centro
3,topografo,por obra,2013-02-23,2013-02-28,1,1-3 años,0-50 horas,jornada ordinaria (8 horas),$501-$750,tecnológico superior,ingeniería/técnico,pedro vicente maldonado,indistinto,suroeste
4,asistente contable,tiempo completo,2013-02-25,2013-03-27,1,1-3 años,0-50 horas,jornada ordinaria (8 horas),$400-$500,tercer nivel,economía/contabilidad,guayaquil,indistinto,sur


## 2. Exploración inicial y limpieza básica

In [69]:
df.shape
# Dimensiones del DataFrame
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

# Información general
df.info()

Filas: 210812, Columnas: 14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210812 entries, 0 to 210811
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   cargo             210812 non-null  object
 1   modo              210812 non-null  object
 2   fechaPublicado    210812 non-null  object
 3   fechaFin          210812 non-null  object
 4   plazas            210812 non-null  int64 
 5   experiencia       210812 non-null  object
 6   capacitacion      210812 non-null  object
 7   jornadas          210812 non-null  object
 8   remuneracion      210812 non-null  object
 9   nivelInstruccion  210812 non-null  object
 10  areaEstudios      210812 non-null  object
 11  ciudad            210812 non-null  object
 12  parroquia         210812 non-null  object
 13  sector            210812 non-null  object
dtypes: int64(1), object(13)
memory usage: 22.5+ MB


Verificar valores nulos

In [70]:
# Conteo de valores nulos por columna
df.isnull().sum()


cargo               0
modo                0
fechaPublicado      0
fechaFin            0
plazas              0
experiencia         0
capacitacion        0
jornadas            0
remuneracion        0
nivelInstruccion    0
areaEstudios        0
ciudad              0
parroquia           0
sector              0
dtype: int64

Eliminar columnas con más del 50% de valores nulos (si aplica)

In [71]:
# Umbral del 50%
umbral = len(df) * 0.5
df = df.dropna(axis=1, thresh=umbral)

# Confirmamos nueva forma
print(f"DataFrame después de eliminar columnas con muchos nulos: {df.shape}")


DataFrame después de eliminar columnas con muchos nulos: (210812, 14)


## 3. Limpieza de columnas clave
Este paso se enfocará en dejar listas las columnas importantes para el análisis.
Asumiremos que entre las columnas que más importan están las relacionadas con el cargo, ubicación y remuneración.

In [72]:
# Ver las primeras filas y nombres de columnas
df.head(3)

,cargo,modo,fechaPublicado,fechaFin,plazas,experiencia,capacitacion,jornadas,remuneracion,nivelInstruccion,areaEstudios,ciudad,parroquia,sector
0,desarrollador java jee,tiempo completo,2013-02-22,2013-03-24,1,sin experiencia,0-50 horas,jornada ordinaria (8 horas),$501-$750,tercer nivel,informática software,quito,indistinto,norte
1,guardias de seguridad con experiencia 1 año,tiempo completo,2013-02-23,2013-02-28,20,7-12 meses,0-50 horas,jornada ordinaria (8 horas),$400-$500,bachiller,recursos humanos/personal,ibarra,indistinto,centro
2,vendedor,tiempo completo,2013-02-23,2013-02-26,2,1-3 años,0-50 horas,jornada ordinaria (8 horas),$400-$500,bachiller,ventas al consumidor,guayaquil,indistinto,centro


In [73]:
df.columns

Index(['cargo', 'modo', 'fechaPublicado', 'fechaFin', 'plazas', 'experiencia',
       'capacitacion', 'jornadas', 'remuneracion', 'nivelInstruccion',
       'areaEstudios', 'ciudad', 'parroquia', 'sector'],
      dtype='object')

Renombrar columnas para trabajar mejor, Esto es útil si los nombres tienen espacios, tildes o caracteres raros

In [74]:
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")
df.columns

Index(['cargo', 'modo', 'fechapublicado', 'fechafin', 'plazas', 'experiencia',
       'capacitacion', 'jornadas', 'remuneracion', 'nivelinstruccion',
       'areaestudios', 'ciudad', 'parroquia', 'sector'],
      dtype='object')

Convertir columna de remuneración a tipo numérico

Primero, identificamos si la columna de remuneración tiene comas, símbolos de dólar o espacios.

In [77]:
# Revisar valores únicos si son pocos o una muestra
print(df["remuneracion"].unique()[:20])

['$501-$750' '$400-$500' '$1001-$1500' '$751-$1000' '$1501-$2000'
 '$2001-$2500' '$2501-$3000' 'más de $ 5001' '$3001-$4000' '$4001-$5000']


Convertir rangos salariales en números, ejemplo: De "$501-$750" → promedio → (501 + 750) / 2 = 625.5

De "más de $ 5001" → usaremos 5001 como base mínima 

Si hay nulos o datos malformados, los convertiremos a NaN

In [78]:
#Esto creará una nueva columna remuneracion_num lista para análisis estadísticos y visualizaciones.
import re
import numpy as np

def parse_remuneracion(value):
    if pd.isnull(value):
        return np.nan

    value = str(value).lower().strip()

    # Caso: rango como "1000-1500"
    rango = re.findall(r"\d+", value)
    if len(rango) == 2:
        return (int(rango[0]) + int(rango[1])) / 2

    # Caso: único valor como "1200"
    elif len(rango) == 1:
        return int(rango[0])

    # No se puede extraer número
    return np.nan


Aplicar la función y crear nueva columna con valores numericos

In [80]:
df["remuneracion_num"] = df["remuneracion"].apply(parse_remuneracion)
print(df[["remuneracion", "remuneracion_num"]].head(10))


  remuneracion  remuneracion_num
0    $501-$750             625.5
1    $400-$500             450.0
2    $400-$500             450.0
3    $501-$750             625.5
4    $400-$500             450.0
5    $400-$500             450.0
6    $400-$500             450.0
7    $400-$500             450.0
8    $400-$500             450.0
9    $501-$750             625.5


## 4. Revisar valores únicos en columnas clave

In [81]:
columnas_categoricas = ["cargo", "modo", "nivelinstruccion", "ciudad", "sector"]

for col in columnas_categoricas:
    print(f"\n--- {col} ---")
    print(df[col].unique()[:10])  # muestra solo los primeros 10 valores únicos



--- cargo ---
['desarrollador java jee' 'guardias de seguridad con experiencia 1 año'
 'vendedor' 'topografo' 'asistente contable' 'mensajero'
 'guardias de seguridad' 'recepcion' 'guias turisticas'
 'supervisor de promotoria con experiencia']

--- modo ---
['tiempo completo' 'por obra' 'eventual' 'tiempo parcial']

--- nivelinstruccion ---
['tercer nivel' 'bachiller' 'tecnológico superior'
 'educación básica /básica superior' 'secundaria sin finalizar'
 'técnico superior' 'sin instruccion'
 'primaria /básica elemental /básica media' 'cuarto nivel-maestria'
 'cuarto nivel-especialidad']

--- ciudad ---
['quito' 'ibarra' 'guayaquil' 'pedro vicente maldonado' 'cayambe'
 'otavalo' 'lago agrio' 'cuenca' 'zamora' 'pichincha']

--- sector ---
['norte' 'centro' 'suroeste' 'sur' 'noroeste' 'oeste' 'noreste' 'este'
 'sureste']


Esto Convierte a minúsculas
Elimina tildes y caracteres especiales
Quita espacios en blanco iniciales y finales

In [82]:
import unicodedata

def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = texto.strip().lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', errors='ignore').decode('utf-8')
    return texto

for col in columnas_categoricas:
    df[col] = df[col].astype(str).apply(limpiar_texto)
